In [30]:
import numpy as np

In [ ]:
with open('day21-input.txt') as f:
    rules = f.readlines()
rules = [x.strip('\n').split(' => ') for x in rules]

In [94]:
rules_dict = {key:val for [key, val] in rules}

In [34]:
rules[:5]

[['../..', '..#/.#./...'],
 ['#./..', '.../#../.##'],
 ['##/..', '.##/###/##.'],
 ['.#/#.', '#.#/..#/#.#'],
 ['##/#.', '.../.##/...']]

In [98]:
rules_dict.get('../..')

'..#/.#./...'

In [12]:
start = ".#./..#/###"

Define all the rotation for the input rules, so we can look it up directly afterwards.

In [49]:
def to_array(string):
    return np.array([list(x) for x in string.split('/')])

In [72]:
start_array = to_array(start)
start_array

array([['.', '#', '.'],
       ['.', '.', '#'],
       ['#', '#', '#']],
      dtype='|S1')

In [70]:
def to_string(array):
    return "/".join("".join(x) for x in array.tolist())

In [71]:
assert to_string(start_array) == start

In [118]:
def all_rot_and_flips(motif):
    ret = list()
    motif_array = to_array(motif)
    ret.append(motif)
    ret.append(to_string(np.rot90(motif_array)))
    ret.append(to_string(np.rot90(motif_array, k=2)))
    ret.append(to_string(np.rot90(motif_array, k=3)))
    ret.append(to_string(np.fliplr(motif_array)))
    ret.append(to_string(np.flipud(motif_array)))
    ret.append(to_string(np.fliplr(np.rot90(motif_array))))
    ret.append(to_string(np.fliplr(np.rot90(motif_array, k=3))))
    return ret

In [119]:
for l in all_rot_and_flips(start):
    print(l.replace('/', '\n'))
    print("")

.#.
..#
###

.##
#.#
..#

###
#..
.#.

#..
#.#
##.

.#.
#..
###

###
..#
.#.

##.
#.#
#..

..#
#.#
.##



In [109]:
def all_rules(rules_dict):
    rules = dict()
    for rule, trans in rules_dict.items():
        for new_rule in all_rot_and_flips(rule):
            rules[new_rule] = trans
    return rules

In [120]:
all_rules_dict = all_rules(rules_dict)

In [138]:
def augment_grid(old_grid=start_array, rules_dict=all_rules_dict):
    new_array = to_array(rules_dict[to_string(old_grid)])
    return new_array

In [139]:
augment_grid()

array([['#', '#', '.', '#'],
       ['#', '#', '.', '#'],
       ['#', '.', '#', '.'],
       ['#', '#', '.', '.']],
      dtype='|S1')

In [172]:
def split_array(old_array):
    a, _ = np.shape(old_array)
    if a == 2 or a == 3:
        return augment_grid(old_array)
    if a % 2 == 0:
        b = a / 2
        new_grid = np.empty((b * 3, b * 3), dtype='|S1')
        for i in range(b):
            for j in range(b):
                new_grid[3*i:3*i+3, 3*j:3*j+3] = split_array(old_array[2*i:2*i+2, 2*j:2*j+2])
        return new_grid
    if a % 3 == 0:
        b = a / 3
        new_grid = np.empty((b * 4, b * 4), dtype='|S1')
        for i in range(b):
            for j in range(b):
                new_grid[4*i:4*i+4, 4*j:4*j+4] = split_array(old_array[3*i:3*i+3, 3*j:3*j+3])
        return new_grid

In [174]:
def n_split(start, n_split):
    res = start
    for i in range(n_split):
        res = split_array(res)
    return res

In [178]:
to_string(n_split(start_array, 3)).count('#')

47

In [179]:
def count_ones(array):
    res = to_string(array)
    return res.count('#')

In [182]:
count_ones(n_split(start_array, 5))

120